# USGS Earthquake Event API Crawler

In [13]:
import os
import requests
import pandas as pd
from io import StringIO

os.mkdir('data')
pd.set_option('display.max_columns', None)

In [14]:
def crawl_event(event_id, save_json=False):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    try:
        params = {"eventid": event_id, "format": "geojson"}
        r = requests.get(url, params=params)
        
        if r.status_code != 200:
            print(f"✗ HTTP {r.status_code}: {r.text[:200]}")
            return None
        
        data = r.json()

        if "features" in data and data["features"]:
            feature = data["features"][0]
        elif data.get("type") == "Feature":
            feature = data
        else:
            print("Can not found event")
            print(f"Response keys: {data.keys()}")
            return None
        
        props = feature["properties"]
        coords = feature["geometry"]["coordinates"]
        
        result = {
            "id": props.get("id"),
            "time": pd.to_datetime(props.get("time", 0), unit="ms"),
            "place": props.get("place"),
            "mag": props.get("mag"),
            "depth": coords[2],
            "lat": coords[1],
            "lon": coords[0],
            "felt": props.get("felt"),
            "cdi": props.get("cdi"),
            "mmi": props.get("mmi"),
            "alert": props.get("alert"),
            "tsunami": props.get("tsunami"),
            "sig": props.get("sig"),
            "url": props.get("url")
        }
        
        print(f"+ {result['place']} - M{result['mag']}")

        if save_json:
            json_path = f"data/event_{event_id}.json"
            with open(json_path, 'w', encoding='utf-8') as f:
                import json
                json.dump(data, f, indent=2, ensure_ascii=False)
            print(f"+ Đã lưu JSON: {json_path}")
        
        return result
        
    except Exception as e:
        print(f"Lỗi: {e}")
        import traceback
        traceback.print_exc()
        return None

In [15]:
def crawl_multiple_events(event_ids, save_all_json=False):
    results = []
    
    for i, event_id in enumerate(event_ids, 1):
        print(f"\n[{i}/{len(event_ids)}] Crawling {event_id}...")
        result = crawl_event(event_id, save_json=save_all_json)
        if result:
            results.append(result)
    
    return results

# Lấy event_id 2023
params = {
    "format": "csv",
    "starttime": "2023-01-01",
    "endtime": "2023-12-31",
    "minmagnitude": 6.0,
    "limit": 10
}
r = requests.get("https://earthquake.usgs.gov/fdsnws/event/1/query", params=params)
df = pd.read_csv(StringIO(r.text))

print(f"Total {len(df)} event:")
print(df[['time', 'id', 'place', 'mag']].to_string(index=False))

Total 10 event:
                    time         id                               place  mag
2023-12-30T17:16:23.833Z us6000m0n6    146 km WSW of Abepura, Indonesia  6.3
2023-12-28T09:15:16.131Z us6000m05c       115 km SE of Kuril’sk, Russia  6.5
2023-12-22T17:36:32.014Z us7000lkxb                     south of Africa  6.1
2023-12-21T14:55:56.291Z usd0010b4f           116 km SE of Adak, Alaska  6.1
2023-12-20T12:11:21.484Z us7000lkb1               11 km E of Iray, Peru  6.2
2023-12-11T06:33:32.480Z us7000li15   174 km NW of Fangale’ounga, Tonga  6.2
2023-12-07T12:56:30.184Z us7000lgwp        118 km S of Isangel, Vanuatu  7.1
2023-12-03T19:49:35.739Z us7000lfxx 34 km ENE of Aras-asan, Philippines  6.9
2023-12-03T14:35:56.830Z us7000lfwj   58 km E of Marihatag, Philippines  6.0
2023-12-03T10:35:52.871Z us7000lftp  47 km ENE of Hinatuan, Philippines  6.6


In [16]:
# Crawl tất cả events
all_results = crawl_multiple_events(df['id'].tolist(), save_all_json=True)


[1/10] Crawling us6000m0n6...
+ 146 km WSW of Abepura, Indonesia - M6.3
+ Đã lưu JSON: data/event_us6000m0n6.json

[2/10] Crawling us6000m05c...
+ 115 km SE of Kuril’sk, Russia - M6.5
+ Đã lưu JSON: data/event_us6000m05c.json

[3/10] Crawling us7000lkxb...
+ south of Africa - M6.1
+ Đã lưu JSON: data/event_us7000lkxb.json

[4/10] Crawling usd0010b4f...
+ 116 km SE of Adak, Alaska - M6.1
+ Đã lưu JSON: data/event_usd0010b4f.json

[5/10] Crawling us7000lkb1...
+ 11 km E of Iray, Peru - M6.2
+ Đã lưu JSON: data/event_us7000lkb1.json

[6/10] Crawling us7000li15...
+ 174 km NW of Fangale’ounga, Tonga - M6.2
+ Đã lưu JSON: data/event_us7000li15.json

[7/10] Crawling us7000lgwp...
+ 118 km S of Isangel, Vanuatu - M7.1
+ Đã lưu JSON: data/event_us7000lgwp.json

[8/10] Crawling us7000lfxx...
+ 34 km ENE of Aras-asan, Philippines - M6.9
+ Đã lưu JSON: data/event_us7000lfxx.json

[9/10] Crawling us7000lfwj...
+ 58 km E of Marihatag, Philippines - M6
+ Đã lưu JSON: data/event_us7000lfwj.json

[10

In [17]:
# Lưu kết quả ra file CSV
if all_results:
    results_df = pd.DataFrame(all_results)
    csv_path = "data/earthquake_events.csv"
    results_df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"Saved CSV: {csv_path}")

Saved CSV: data/earthquake_events.csv
